# 1. Introduction

In this notebook, we will create a proof-on-concept web application that shows the finally selected model in accion. The application will be created using the [Gradio](https://www.gradio.app/) framework and then deployed to HuggingFace to make it publicly available for all.

# 2. Importing Libraries

In [56]:
import os
import json
import shutil
from pathlib import Path

# 3. Creating Directories and Files

In [57]:
# Create FoodVision demo path
food_vision_demo_path = Path("../demo")  # Path joining with Path objects
food_vision_demo_path.mkdir(parents=True, exist_ok=True)

# Make examples directory
food_vision_demo_examples_path = food_vision_demo_path / "examples"
food_vision_demo_examples_path.mkdir(parents=True, exist_ok=True)

# Specify model directory
food_vision_model_path = Path("../models")

## 3.1. Food Description File

In [58]:
# Create a JSON file with the description of th 101 types of food
food_descriptions = {
    "apple_pie": "A classic American dessert made with spiced apple filling encased in a flaky pastry crust, often served with ice cream.",
    "baby_back_ribs": "Juicy, tender pork ribs, slow-cooked and coated with a flavorful barbecue sauce.",
    "baklava": "A rich, sweet pastry layered with chopped nuts and honey or syrup, originating from the Middle East and Mediterranean regions.",
    "beef_carpaccio": "Thinly sliced raw beef served with olive oil, lemon, capers, and Parmesan, often as a starter.",
    "beef_tartare": "Finely chopped or ground raw beef mixed with seasonings like capers, onions, and egg yolk, served uncooked.",
    "beet_salad": "A refreshing dish featuring cooked or raw beets paired with greens, goat cheese, nuts, and a tangy vinaigrette.",
    "beignets": "Deep-fried dough pastries dusted with powdered sugar, popular in New Orleans as a sweet treat.",
    "bibimbap": "A Korean mixed rice dish topped with vegetables, meat, an egg, and spicy gochujang sauce.",
    "bread_pudding": "A comforting dessert made from bread soaked in a sweet custard mixture and baked until golden.",
    "breakfast_burrito": "A hearty wrap filled with scrambled eggs, cheese, meats, and sometimes vegetables or beans.",
    "bruschetta": "An Italian appetizer of toasted bread rubbed with garlic and topped with fresh tomatoes, olive oil, and basil.",
    "caesar_salad": "A classic salad made with romaine lettuce, croutons, Parmesan cheese, and Caesar dressing.",
    "cannoli": "An Italian dessert consisting of a crispy pastry shell filled with sweet ricotta cheese and sometimes chocolate chips.",
    "caprese_salad": "A simple Italian salad of fresh mozzarella, tomatoes, and basil, drizzled with olive oil and balsamic vinegar.",
    "carrot_cake": "A moist cake made with grated carrots and often topped with cream cheese frosting.",
    "ceviche": "A seafood dish where fresh fish or shrimp is 'cooked' in citrus juice and mixed with onions, peppers, and cilantro.",
    "cheese_plate": "An assortment of cheeses served with accompaniments like crackers, fruit, and nuts.",
    "cheesecake": "A creamy dessert made with a mixture of cream cheese, sugar, and eggs on a graham cracker or pastry crust.",
    "chicken_curry": "A flavorful dish of chicken cooked in a spicy, aromatic curry sauce, popular in South Asian cuisine.",
    "chicken_quesadilla": "A Mexican dish of tortillas filled with cheese, chicken, and spices, grilled until crispy and melty.",
    "chicken_wings": "Juicy chicken wings fried or baked and often coated with a tangy or spicy sauce.",
    "chocolate_cake": "A rich, moist dessert made with layers of chocolate sponge and frosting.",
    "chocolate_mousse": "A light and creamy dessert made with whipped cream and melted chocolate, often served chilled.",
    "churros": "Fried dough pastry, often coated in sugar and cinnamon, typically served with chocolate dipping sauce.",
    "clam_chowder": "A creamy soup made with clams, potatoes, and onions, often served in a bread bowl.",
    "club_sandwich": "A layered sandwich filled with turkey, bacon, lettuce, tomato, and mayonnaise.",
    "crab_cakes": "Pan-fried or baked patties made from lump crab meat and breadcrumbs, seasoned with spices.",
    "creme_brulee": "A custard dessert with a caramelized sugar crust on top, served cold.",
    "croque_madame": "A French grilled sandwich filled with ham and cheese, topped with a fried egg.",
    "cup_cakes": "Small, individual-sized cakes often topped with frosting and decorations.",
    "deviled_eggs": "Hard-boiled eggs halved and filled with a seasoned yolk mixture.",
    "donuts": "Sweet, fried dough pastries, often glazed or topped with sprinkles.",
    "dumplings": "Small dough parcels filled with meat or vegetables, steamed, boiled, or fried.",
    "edamame": "Boiled or steamed young soybeans served in their pods, sprinkled with salt.",
    "eggs_benedict": "A breakfast dish of poached eggs, English muffins, Canadian bacon, and hollandaise sauce.",
    "escargots": "Cooked snails, often served with garlic butter in French cuisine.",
    "falafel": "Deep-fried balls or patties made from ground chickpeas or fava beans, often served in pita bread.",
    "filet_mignon": "A tender and juicy cut of beef steak, often served as a fine dining dish.",
    "fish_and_chips": "A classic British dish of battered and fried fish served with crispy fries.",
    "foie_gras": "A French delicacy made from the liver of a fattened duck or goose, often served as a pâté.",
    "french_fries": "Thinly sliced potatoes deep-fried until crispy, often served as a side dish.",
    "french_onion_soup": "A hearty soup made with caramelized onions and beef broth, topped with bread and melted cheese.",
    "french_toast": "Bread slices soaked in a milk and egg mixture, fried until golden, and served with syrup.",
    "fried_calamari": "Lightly battered and fried squid rings, often served with marinara or aioli sauce.",
    "fried_rice": "A stir-fried rice dish mixed with vegetables, eggs, and meat or seafood.",
    "frozen_yogurt": "A creamy, tangy dessert similar to ice cream but made with yogurt.",
    "garlic_bread": "Toasted bread slices flavored with garlic butter, often served as a side dish.",
    "gnocchi": "Soft Italian dumplings made from potato, flour, and sometimes cheese, served with sauce.",
    "greek_salad": "A fresh salad made with cucumbers, tomatoes, olives, feta cheese, and olive oil.",
    "grilled_cheese_sandwich": "A hot sandwich filled with melted cheese, grilled until crispy.",
    "grilled_salmon": "Salmon fillets cooked on a grill, often seasoned with herbs and spices.",
    "guacamole": "A creamy dip made from mashed avocados, lime juice, onions, and cilantro.",
    "gyoza": "Japanese dumplings filled with minced meat and vegetables, pan-fried and steamed.",
    "hamburger": "A ground beef patty served in a bun with toppings like lettuce, tomato, and cheese.",
    "hot_and_sour_soup": "A Chinese soup with a tangy and spicy flavor, filled with tofu, mushrooms, and vegetables.",
    "hot_dog": "A grilled or boiled sausage served in a bun with condiments like ketchup and mustard.",
    "huevos_rancheros": "A Mexican breakfast dish with fried eggs, tortillas, and salsa.",
    "hummus": "A creamy dip made from blended chickpeas, tahini, lemon juice, and garlic.",
    "ice_cream": "A frozen dessert made from sweetened cream, often flavored and served in scoops.",
    "lasagna": "An Italian pasta dish layered with meat, cheese, and tomato sauce, baked until bubbly.",
    "lobster_bisque": "A creamy soup made from lobster stock, often served as a gourmet dish.",
    "lobster_roll_sandwich": "A sandwich filled with chunks of lobster meat, dressed in butter or mayo.",
    "macaroni_and_cheese": "A comfort food dish of macaroni pasta mixed with a creamy cheese sauce.",
    "macarons": "Delicate French almond cookies filled with flavored buttercream or ganache.",
    "miso_soup": "A Japanese soup made with fermented soybean paste, tofu, and seaweed.",
    "mussels": "Shellfish steamed and served in a flavorful broth, often with bread.",
    "nachos": "Tortilla chips topped with melted cheese and various toppings like jalapeños and sour cream.",
    "omelette": "Beaten eggs cooked in a frying pan, often filled with cheese, meats, or vegetables.",
    "onion_rings": "Crispy, deep-fried onion slices, often served as a side dish or snack.",
    "oysters": "Shellfish often served raw on the half shell with lemon or hot sauce.",
    "pad_thai": "Stir-fried rice noodles with eggs, tofu or shrimp, peanuts, and a tamarind-based sauce.",
    "paella": "Spanish rice dish with seafood, meats, and saffron, traditionally cooked in a single pan.",
    "pancakes": "Fluffy round cakes made from flour, eggs, and milk, served with syrup and butter.",
    "panna_cotta": "Italian dessert of sweetened cream thickened with gelatin, often topped with berries or caramel.",
    "peking_duck": "Crispy duck served with pancakes, hoisin sauce, and sliced scallions.",
    "pho": "Vietnamese noodle soup with broth, rice noodles, herbs, and beef or chicken.",
    "pizza": "Flat dough topped with tomato sauce, cheese, and various toppings, baked until crispy.",
    "pork_chop": "Grilled or roasted cut of pork, often served with vegetables or a flavorful sauce.",
    "poutine": "Canadian dish with French fries, cheese curds, and gravy.",
    "prime_rib": "Tender beef rib roast, often seasoned and slow-cooked, served for special occasions.",
    "pulled_pork_sandwich": "Slow-cooked shredded pork with barbecue sauce, served on a bun.",
    "ramen": "Japanese noodle soup with broth, wheat noodles, and toppings like eggs and pork.",
    "ravioli": "Italian dumplings filled with cheese, meat, or vegetables, served with sauce.",
    "red_velvet_cake": "Rich cake with a hint of cocoa and red coloring, layered with cream cheese frosting.",
    "risotto": "Italian creamy rice dish cooked with broth and flavored with vegetables or meat.",
    "samosa": "Deep-fried pastry filled with spiced potatoes, peas, and sometimes meat.",
    "sashimi": "Thinly sliced raw fish or seafood, often served with soy sauce and wasabi.",
    "scallops": "Delicate shellfish, often seared and served as a seafood entrée.",
    "seaweed_salad": "Salad made from seasoned edible seaweed, typically with sesame oil and soy sauce.",
    "shrimp_and_grits": "Southern dish of shrimp cooked in a savory sauce, served with creamy grits.",
    "spaghetti_bolognese": "Italian pasta with a rich meat sauce made from ground beef, tomatoes, and herbs.",
    "spaghetti_carbonara": "Pasta with a creamy sauce made from eggs, cheese, pancetta, and black pepper.",
    "spring_rolls": "Rice paper rolls filled with vegetables, meat, or seafood, served with dipping sauce.",
    "steak": "Grilled or pan-seared beef cut, often served with vegetables or potatoes.",
    "strawberry_shortcake": "Dessert with layers of cake, fresh strawberries, and whipped cream.",
    "sushi": "Japanese dish with vinegared rice, raw or cooked seafood, and sometimes vegetables.",
    "tacos": "Mexican dish with a tortilla filled with meat, vegetables, cheese, and salsa.",
    "takoyaki": "Japanese snack made from batter, octopus, and tempura bits, served with takoyaki sauce.",
    "tiramisu": "Italian dessert with coffee-soaked ladyfingers, mascarpone cheese, and cocoa powder.",
    "tuna_tartare": "Finely diced raw tuna, often mixed with soy sauce and served as an appetizer.",
    "waffles": "Batter-based dish cooked in a grid pattern, served with syrup, fruit, or whipped cream.",
    "unknown": "No sufficient confidence to classify the image." # "The current picture is not recognized as one of the 101 food classes. Please try again."
}

food_descriptions = {key.replace("_", " "): value for key, value in food_descriptions.items()}

# Save to JSON file
food_descriptions_json = os.path.join(food_vision_demo_path, 'food_descriptions.json')
with open(food_descriptions_json, 'w') as f:
    json.dump(food_descriptions, f, indent=4)

# Open JSON file
# Load the food descriptions from the JSON file
with open(food_descriptions_json, 'r') as f:
    food_descriptions = json.load(f)


## 3.2 Class Name File

In [59]:
# Create path to Food-101 class names
class_names = list(food_descriptions.keys())
food_vision_class_names_path = food_vision_demo_path / "class_names.txt"

# Write Food-101 class names list to file
with open(food_vision_class_names_path, "w") as f:
    print(f"[INFO] Saving Food-101 class names to {food_vision_class_names_path}")
    f.write("\n".join(class_names)) # leave a new line between each class

# Open Food-101 class names file
with open(food_vision_class_names_path, "r") as f:
    print(f"[INFO] Reading Food-101 class names from {food_vision_class_names_path}")
    class_names = f.read().splitlines()

 # Show the first 5 class names
class_names[:5]

[INFO] Saving Food-101 class names to ..\demo\class_names.txt
[INFO] Reading Food-101 class names from ..\demo\class_names.txt


['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']

## 3.3. Creating Sample Images

In [60]:
# Create an examples directory
food_vision_examples_path = food_vision_demo_path / "examples"
food_vision_examples_path.mkdir(parents=True, exist_ok=True)

# Collect five test image paths
food_vision_examples = [Path('../data/food-101_100_percent/test/sushi/511665.jpg'),
                        Path('../data/food-101_100_percent/test/hot_and_sour_soup/1134579.jpg'),
                        Path('../data/food-101_100_percent/test/paella/2083247.jpg'),
                        Path('../data/food-101_100_percent/test/carrot_cake/470617.jpg'),
                        Path('../data/food-101_100_percent/test/pizza/3770514.jpg'),
                        Path('../data/food-101_100_percent/test/pork_chop/137031.jpg'),
                        Path('../data/food-101_100_percent/test/steak/1498778.jpg'),
                        Path('../data/food-101_100_percent/test/fried_calamari/936686.jpg')
                        ]

# Copy the images to the examples directory
for example in food_vision_examples:
    example_path = food_vision_examples_path / example.name
    if not example_path.exists():
        print(f"[INFO] Copying {example} to {example_path}")
        shutil.copy(src=example, dst=example_path)

## 3.4. Creating the ViT Model

In [61]:
%%writefile ../demo/model.py
import torch
import torchvision
from pathlib import Path
from vision_transformer import ViT

def load_model(model: torch.nn.Module,
               model_weights_dir: str,
               model_weights_name: str):

    """Loads a PyTorch model from a target directory.

    Args:
    model: A target PyTorch model to load.
    model_weights_dir: A directory where the model is located.
    model_weights_name: The name of the model to load.
      Should include either ".pth" or ".pt" as the file extension.

    Example usage:
    model = load_model(model=model,
                       model_weights_dir="models",
                       model_weights_name="05_going_modular_tingvgg_model.pth")

    Returns:
    The loaded PyTorch model.
    """
    # Create the model directory path
    model_dir_path = Path(model_weights_dir)

    # Create the model path
    assert model_weights_name.endswith(".pth") or model_weights_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_path = model_dir_path / model_weights_name

    # Load the model
    print(f"[INFO] Loading model from: {model_path}")
    
    model.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('cpu')))
    
    return model

def create_vitbase_model(
    model_weights_dir:Path,
    model_weights_name:str,
    img_size:int=224,
    num_classes:int=101,
    compile:bool=False
    ):
    """
    Creates a ViT-B/16 model with the specified number of classes.

    Args:
        model_weights_dir: A directory where the model is located.
        model_weights_name: The name of the model to load.
        img_size: The size of the input image.
        num_classes: The number of classes for the classification task.

    Returns:
    The created ViT-B/16 model.
    """    
    # Instantiate the model
    vitbase16_model = ViT(
        img_size=img_size,
        in_channels=3,
        patch_size=16,
        num_transformer_layers=12,
        emb_dim=768,
        mlp_size=3072,
        num_heads=12,
        attn_dropout=0,
        mlp_dropout=0.1,
        emb_dropout=0.1,
        num_classes=num_classes
    )
    
    # Compile the model
    if compile:
        vitbase16_model = torch.compile(vitbase16_model, backend="aot_eager")

    # Load the trained weights
    vitbase16_model = load_model(
        model=vitbase16_model,
        model_weights_dir=model_weights_dir,
        model_weights_name=model_weights_name
        )
    
    return vitbase16_model

# Create an EfficientNet-B0 Model
def create_effnetb0(
        model_weights_dir: Path,
        model_weights_name: str,
        num_classes: int=2,
        dropout: float=0.2
        ):
    """Creates an EfficientNetB0 feature extractor model and transforms.

    Args:
        model_weights_dir: A directory where the model is located.
        model_weights_name: The name of the model to load.
        num_classes (int, optional): number of classes in the classifier head.
        dropout (float, optional): Dropout rate. Defaults to 0.2.

    Returns:
        effnetb0_model (torch.nn.Module): EffNetB0 feature extractor model.
        transforms (torchvision.transforms): Image transforms.
    """
    
    # Load pretrained weights
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights 
    effnetb0_model = torchvision.models.efficientnet_b0(weights=weights).to('cpu')

    # Recreate the classifier layer and seed it to the target device
    effnetb0_model.classifier = torch.nn.Sequential(
        torch.nn.Dropout(p=dropout, inplace=True), 
        torch.nn.Linear(in_features=1280, 
                        out_features=num_classes,
                        bias=True))
    
    # Create the model directory path
    model_dir_path = Path(model_weights_dir)

    # Create the model path
    assert model_weights_name.endswith(".pth") or model_weights_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_path = model_dir_path / model_weights_name

    # Load the state dictionary into the model
    effnetb0_model.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('cpu')))
        
    return effnetb0_model

Overwriting ../demo/model.py


In [62]:
# Copy required libraries
shutil.copy(src="modules/vision_transformer.py", dst="../demo/")

'../demo/vision_transformer.py'

## 3.5. Creating the App

In [63]:
%%writefile ../demo/app.py

# Imports and class names setup
import os
import torch
import json
import gradio as gr
from model import create_vitbase_model, create_effnetb0
from timeit import default_timer as timer
from typing import Tuple, Dict
from torchvision.transforms import v2


# Specify class names
food_vision_class_names_path = "class_names.txt"
with open(food_vision_class_names_path, "r") as f:
    class_names = f.read().splitlines()

# Specify number of classes
num_classes = len(class_names) - 1 # 101, "unknown" to be discarded

# Load the food description file
food_descriptions_json = "food_descriptions.json"
with open(food_descriptions_json, 'r') as f:
    food_descriptions = json.load(f)

# Instantiate the model
classification_model_name_path = "effnetb0_classif_epoch13.pth"
effnetb0_model = create_effnetb0(
    model_weights_dir=".",
    model_weights_name=classification_model_name_path,
    num_classes=2
    )

# Load the ViT-Base/16 transformer with input image of 224x224 pixels
vitbase_model_1 = create_vitbase_model(
    model_weights_dir=".",
    model_weights_name="vitbase16_5.pth",
    img_size=224,
    num_classes=num_classes,
    compile=False
)

# Specify manual transforms for model_1
transforms_1 = v2.Compose([    
    v2.Resize((242, 242)),
    v2.CenterCrop((224, 224)),    
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]) 
])

# Load the ViT-Base/16 transformer with input image of 384x384 pixels 
vitbase_model_2 = create_vitbase_model(
    model_weights_dir=".",
    model_weights_name="vitbase16_2_2024-12-31.pth",
    img_size=384,
    num_classes=num_classes,
    compile=True
)

# Specify manual transforms for model_2
transforms_2 = v2.Compose([    
    v2.Resize(384), #v2.Resize((384, 384)),
    v2.CenterCrop((384, 384)),    
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]) 
])

# Put models into evaluation mode and turn on inference mode
effnetb0_model.eval()
vitbase_model_1.eval()
vitbase_model_2.eval()

# Specify default ViT model
default_model = "Vision Transformer - 384x384 pixels (higher accuracy, slower predictions)" # "Vision Transformer - 224x224 pixels (lower accuracy, faster predictions)"

# Predict function
def predict(image) -> Tuple[Dict, str, str]:

    """Transforms and performs a prediction on image and returns prediction and time taken.
    """
    try:
        # Start the timer
        start_time = timer()

        # Select the appropriate model based on the user's choice
        if default_model == "Vision Transformer - 384x384 pixels (higher accuracy, slower predictions)":
            vitbase_model = vitbase_model_2
            transforms = transforms_2
        else:
            vitbase_model = vitbase_model_1
            transforms = transforms_1
        
        # Transform the target image and add a batch dimension
        image = transforms(image).unsqueeze(0)
        
        # Make prediction...
        with torch.inference_mode():

            # If the picture is food
            if effnetb0_model(image)[:,1].cpu() >= 0.9981166124343872:

                # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
                pred_probs = torch.softmax(vitbase_model(image), dim=1) # 101 classes

                # Calculate entropy
                entropy = -torch.sum(pred_probs * torch.log(pred_probs), dim=1).item()

                # Create a prediction label and prediction probability dictionary for each prediction class
                pred_classes_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(num_classes)}
                pred_classes_and_probs["unknown"] = 0.0

                # Get the top predicted class
                top_class = max(pred_classes_and_probs, key=pred_classes_and_probs.get)

                # If the image is likely to be an unknown category
                if pred_probs[0][class_names.index(top_class)] <= 0.5 and entropy > 2.6:

                    # Create prediction label and prediction probability for class unknown and rescale the rest of predictions
                    pred_classes_and_probs["unknown"] = pred_probs.max() * 1.25
                    prob_sum = sum(pred_classes_and_probs.values())
                    pred_classes_and_probs = {key: value / prob_sum for key, value in pred_classes_and_probs.items()}

                    # Get the top predicted class
                    top_class = "unknown"

            # Otherwise
            else:

                # Set all probabilites to zero except class unknown
                pred_classes_and_probs = {class_names[i]: 0.0 for i in range(num_classes)}
                pred_classes_and_probs["unknown"] = 1.0
            
                # Get the top predicted class
                top_class = "unknown"
        
        # Get the description of the top predicted class
        top_class_description = food_descriptions.get(top_class, "Description not available.")

        # Calculate the prediction time
        pred_time = f"{round(timer() - start_time, 1)} s."
        
        # Return the prediction dictionary and prediction time 
        return pred_classes_and_probs, pred_time, top_class_description
    
    except Exception as e:
        print(f"[ERROR] {e}")
        return {}, "Error during prediction.", "N/A"

# Configure and design the Gradio App

# Create title, description, and examples
title = "Transform-Eats Large<br>🥪🥗🥣🥩🍝🍣🍰"
description = f"""
A cutting-edge Vision Transformer (ViT) model to classify 101 delicious food types. Discover the power of AI in culinary recognition.

### Supported Food Types
{', '.join(class_names[:-1])}.
"""

# Configure the upload image area
upload_input = gr.Image(type="pil", label="Upload Image", sources=['upload'], show_label=True, mirror_webcam=False)

# Configure the dropdown option
#model_dropdown = gr.Dropdown(
#    choices=["Vision Transformer - 384x384 pixels (higher accuracy, slower predictions)",
#             "Vision Transformer - 224x224 pixels (lower accuracy, faster predictions)"],
#    value="Vision Transformer - 384x384 pixels (higher accuracy, slower predictions)",
#    label="Select Model:"
#)

# Configure the sample image area
food_vision_examples = [["examples/" + example] for example in os.listdir("examples")]

# Author
article = "Created by Sergio Sanz."

# Create sliders for the thresholds
#prob = gr.Slider(minimum=0, maximum=1, step=0.05, value=0.4, label="Probability Threshold")
#entropy = gr.Slider(minimum=0, maximum=4.615, step=0.5, value=2.5, label="Entropy Threshold")

# Create the Gradio demo
demo = gr.Interface(fn=predict,                                                # mapping function from input to outputs
                    inputs=upload_input,                                       # inputs #[upload_input, model_dropdown]
                    outputs=[gr.Label(num_top_classes=3, label="Prediction"), 
                             gr.Textbox(label="Prediction time:"),
                             gr.Textbox(label="Food Description:")],           # outputs
                    examples=food_vision_examples,                             # Create examples list from "examples/" directory
                    cache_examples=True,                                       # Cache the examples
                    title=title,                                               # Title of the app
                    description=description,                                   # Brief description of the app
                    article=article,                                           # Created by...
                    theme="ocean")                                             # Theme

# Launch the demo!
demo.launch()

Overwriting ../demo/app.py


## 3.6. Creating a Requirements File

In [64]:
%%writefile ../demo/requirements.txt
torch==2.5.0
torchvision==0.20.0
gradio==5.7.1

Overwriting ../demo/requirements.txt


# 3.7. Copy the Models to the Repository

In [65]:
# Binary classification model: food vs no food
source = food_vision_model_path / "effnetb0_classif_epoch13.pth"
destination = food_vision_demo_path
print(f"[INFO] Copying {source} to {destination}")
shutil.copy(src=source, dst=destination)

# 101-class classification model 1
source = food_vision_model_path / "vitbase16_5.pth"
destination = food_vision_demo_path
print(f"[INFO] Copying {source} to {destination}")
shutil.copy(src=source, dst=destination)

# 101-class classification model 2
source = food_vision_model_path / "vitbase16_2_2024-12-31.pth"
destination = food_vision_demo_path
print(f"[INFO] Copying {source} to {destination}")
shutil.copy(src=source, dst=destination)

[INFO] Copying ..\models\effnetb0_classif_epoch13.pth to ..\demo
[INFO] Copying ..\models\vitbase16_5.pth to ..\demo
[INFO] Copying ..\models\vitbase16_2_2024-12-31.pth to ..\demo


'..\\demo\\vitbase16_2_2024-12-31.pth'

In [66]:
import gradio as gr

gr.themes.builder()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
